# BERT

In [109]:
from datasets import Dataset
import pandas as pd
from transformers import BertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
import torch

# Load dataset
# datafile = '../data/Bitext.csv'
datafile = './data/bert_input.csv'
df = pd.read_csv(datafile)
df = df[['instruction', 'intent']]
print(len(df))

# Convert to Hugging Face dataset
intent_dataset = Dataset.from_pandas(df)

# get device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(example):
    return tokenizer(example['instruction'], padding='max_length', truncation=True, max_length=128)

# Apply tokenization to the dataset
intent_dataset = intent_dataset.map(tokenize_function, remove_columns=["instruction"])

# Map 'intent' to numerical labels
intent_labels = {intent: idx for idx, intent in enumerate(df['intent'].unique())}
intent_dataset = intent_dataset.map(lambda x: {'labels': intent_labels[x['intent']]})

# Number of intents
num_intents = len(df['intent'].unique())
print(num_intents)

# Load DistilBERT model for sequence classification
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=num_intents)

# Data collator (use for sequence classification, handles padding)
data_collator = DataCollatorWithPadding(tokenizer, padding='max_length', max_length=128)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./intent_model',
    save_strategy="epoch",
    save_total_limit=2,
    num_train_epochs=50,
    per_device_train_batch_size=8,
    logging_dir='./logs',
    logging_steps=100,  # Log every 10 steps
    save_steps=500,
    disable_tqdm=False,
)


# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=intent_dataset,
    data_collator=data_collator,
)

# Train the model with tqdm progress bar
trainer.train()
tokenizer.save_pretrained('./intent_model')



104
Using device: cuda


Map: 100%|██████████| 104/104 [00:00<00:00, 21012.94 examples/s]


7


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
 16%|█▌        | 102/650 [00:13<00:52, 10.43it/s]

{'loss': 0.6105, 'grad_norm': 0.7972972989082336, 'learning_rate': 4.230769230769231e-05, 'epoch': 7.69}


...

 93%|█████████▎| 602/650 [01:25<00:08,  5.51it/s]

{'loss': 0.0021, 'grad_norm': 0.022441180422902107, 'learning_rate': 3.846153846153847e-06, 'epoch': 46.15}


100%|██████████| 650/650 [01:35<00:00,  6.82it/s]

{'train_runtime': 95.3521, 'train_samples_per_second': 54.535, 'train_steps_per_second': 6.817, 'train_loss': 0.09779059088000884, 'epoch': 50.0}


('./intent_model/tokenizer_config.json',
 './intent_model/special_tokens_map.json',
 './intent_model/vocab.txt',
 './intent_model/added_tokens.json')

In [110]:
# store the intent labels
import json
with open('./intent_model/intent_labels.json', 'w') as f:
    json.dump(intent_labels, f)

In [94]:
# Function to test the trained model
def test_model(input_text, model, tokenizer):
    # Tokenize the input text without 'token_type_ids'
    inputs = tokenizer(input_text, return_tensors='pt', padding=True, truncation=True, max_length=128)
    
    # Remove 'token_type_ids' from the input if it exists
    inputs.pop('token_type_ids', None)
    
    # Move tensors to the appropriate device (GPU if available)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)
    inputs = {key: value.to(device) for key, value in inputs.items()}
    
    # Make prediction
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Get predicted label (the class with the highest score)
    logits = outputs.logits
    predicted_class_id = torch.argmax(logits, dim=-1).item()
    
    return predicted_class_id

# Test the trained model with a sample input
input_text = "[INT] [BOT] [USR] Hey i want to know list my orders"
predicted_class_id = test_model(input_text, model, tokenizer)

# Print the predicted intent
print(f"Predicted intent ID: {predicted_class_id}")

# Optionally map the predicted intent ID back to the intent label
# get key from value
predicted_intent = list(intent_labels.keys())[list(intent_labels.values()).index(predicted_class_id)]
print(f"Predicted intent: {predicted_intent}")


Predicted intent ID: 2
Predicted intent: list_orders


In [95]:
df.intent.unique()

array(['track_order', 'give_order_id', 'list_orders',
       'give_list_order_params', 'cancel_order', 'give_reason',
       'confirm_command'], dtype=object)

In [96]:
# Load the model
# model = DistilBertForSequenceClassification.from_pretrained('./results/checkpoint-1500')

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Test the trained model with a sample input
input_text = "[INT] track_order [BOT] Please share the order ID [USR] 1231"
print(len(input_text))

# Get the predicted intent ID
predicted_class_id = test_model(input_text, model, tokenizer)

# Print the predicted intent
print(f"Predicted intent ID: {predicted_class_id}")
print(f"Predicted intent: {list(intent_labels.keys())[list(intent_labels.values()).index(predicted_class_id)]}")

# print probability scores of other intents
def get_intent_probs(input_text, model, tokenizer):
    # Tokenize the input text without 'token_type_ids'
    inputs = tokenizer(input_text, return_tensors='pt', padding=True, truncation=True, max_length=128)
    
    # Remove 'token_type_ids' from the input if it exists
    inputs.pop('token_type_ids', None)
    
    # Move tensors to the appropriate device (GPU if available)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)
    inputs = {key: value.to(device) for key, value in inputs.items()}
    
    # Make prediction
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Get predicted label (the class with the highest score)
    logits = outputs.logits
    probs = torch.softmax(logits, dim=-1)
    
    return probs

# Get the probability scores for all intents
probs = get_intent_probs(input_text, model, tokenizer)

test_df = pd.read_csv(datafile)
test_df = test_df[['instruction', 'intent']]

# randomly take 15 samples
test_df = test_df.sample(15)

# Get the predicted intent ID for each sample
test_df['predicted_intent_id'] = test_df['instruction'].apply(lambda x: test_model(x, model, tokenizer))

# Map the predicted intent ID back to the intent label
test_df['predicted_intent'] = test_df['predicted_intent_id'].apply(lambda x: list(intent_labels.keys())[list(intent_labels.values()).index(x)])

for idx, row in test_df.iterrows():
    print(f"Instruction: {row['instruction']}")
    print(f"True intent: {row['intent']}")
    print(f"Predicted intent: {row['predicted_intent']}")
    print('')


60
Predicted intent ID: 1
Predicted intent: give_order_id
Instruction: [INT] cancel_order [BOT] Can you confirm the cancellation of your most recent order? [USR] No, don’t proceed with it.
True intent: confirm_command
Predicted intent: confirm_command

Instruction: [INT] [BOT]  [USR] Where is my order?
True intent: track_order
Predicted intent: track_order

Instruction: [INT] [BOT]  [USR] Can you check my order 12345?
True intent: track_order
Predicted intent: track_order

Instruction: [INT] [BOT]  [USR] I’d like to know the status of my order.
True intent: track_order
Predicted intent: track_order

Instruction: [INT] cancel_order [BOT] Are you sure you want to cancel this order? [USR] Yes, I am sure.
True intent: confirm_command
Predicted intent: confirm_command

Instruction: [INT] [BOT]  [USR] I want to cancel order 998877.
True intent: cancel_order
Predicted intent: cancel_order

Instruction: [INT] cancel_order [BOT] What’s the reason for canceling this order? [USR] I ordered the wr

# NER

In [97]:
from transformers import BertTokenizerFast
from datasets import Dataset
import pandas as pd

# Initialize tokenizer
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
special_tokens = ["[INT]", "[BOT]", "[USR]"]
tokenizer.add_special_tokens({"additional_special_tokens": special_tokens})


# Load the data
data = pd.read_csv('./data/ner_data.csv')
data["labels"] = data["labels"].apply(eval)

# Extract unique labels and create a mapping
unique_labels = sorted({label for labels in data["labels"] for label in labels})
label_to_id = {label: idx for idx, label in enumerate(unique_labels)}

print("Label to ID Mapping:", label_to_id)

# Convert DataFrame to list of dictionaries for Dataset.from_list
data = data.to_dict(orient="records")

# Tokenize and align labels
def tokenize_and_align_labels(examples):
    # Tokenize the text
    tokenized_inputs = tokenizer(examples["instruction"], truncation=True, is_split_into_words=False)
    tokens = tokenizer.convert_ids_to_tokens(tokenized_inputs["input_ids"])
    word_ids = tokenized_inputs.word_ids()
    previous_word_idx = None
    label_ids = []
    labels = examples["labels"]

    # print(f"{'Token':<15} {'Word':<15} {'Label':<15}")

    for i, word_idx in enumerate(word_ids):
        token = tokens[i]  # Current token

        if word_idx is None:  # Special tokens ([CLS], [SEP], padding)
            word = "N/A"
            label = "Special"
            label_ids.append(-100)
        elif word_idx != previous_word_idx:  # First subword
            word = examples["instruction"][word_idx]
            label = labels[word_idx]
            label_ids.append(label_to_id[label])
        else:  # Subword token
            word = examples["instruction"][word_idx]
            label = labels[word_idx]
            token = f"Subword: {token} (Part of: {word})"
            label_ids.append(-100)

        # Print token, word, and label in the same line
        # print(f"{token:<15} {word:<15} {label:<15}")
        previous_word_idx = word_idx

    tokenized_inputs["labels"] = label_ids
    return tokenized_inputs

# Convert to Hugging Face Dataset
dataset = Dataset.from_list(data)

# Map tokenizer to the dataset
dataset = dataset.map(tokenize_and_align_labels)


Label to ID Mapping: {'B-AFFIRMATION': 0, 'B-CONFIRMATION': 1, 'B-COUNT': 2, 'B-END_DATE': 3, 'B-ORD': 4, 'B-REASON': 5, 'B-START_DATE': 6, 'I-END_DATE': 7, 'I-REASON': 8, 'I-START_DATE': 9, 'O': 10}


Map: 100%|██████████| 104/104 [00:00<00:00, 6982.90 examples/s]


In [115]:
# store the label_to_id mapping
import json
with open('./ner_model/entity_labels.json', 'w') as f:
    json.dump(label_to_id, f)

In [104]:
from transformers import BertTokenizerFast, BertForTokenClassification, TrainingArguments, Trainer
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
import pandas as pd

# Initialize tokenizer
# tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

# Load the data
data = pd.read_csv('./data/ner_data.csv')

# Convert stringified lists in 'labels' to actual lists
data["labels"] = data["labels"].apply(eval)

# Extract unique labels and create a mapping
unique_labels = sorted({label for labels in data["labels"] for label in labels})

# unique_labels = ['B-ORD', 'I-ORD', 'O']
label_to_id = {label: idx for idx, label in enumerate(unique_labels)}
id_to_label = {idx: label for label, idx in label_to_id.items()}

print("Label to ID Mapping:", label_to_id)

# Convert DataFrame to list of dictionaries
data_dict = data.to_dict(orient="records")


# Tokenize and align labels

# Convert data to Hugging Face Dataset
hf_dataset = Dataset.from_list(data_dict)

# Split the dataset into train and validation sets
train_data, val_data = train_test_split(data_dict, test_size=0.1, random_state=42)
hf_dataset = DatasetDict({
    "train": Dataset.from_list(train_data),
    "validation": Dataset.from_list(val_data),
})

# Apply the tokenizer and label alignment
hf_dataset = hf_dataset.map(tokenize_and_align_labels, batched=False)
# Define the model
model = BertForTokenClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=len(label_to_id),
    id2label=id_to_label,
    label2id=label_to_id
)

model.resize_token_embeddings(len(tokenizer)) # need to test

# Define training arguments
training_args = TrainingArguments(
    output_dir="./ner_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,  # Lower the learning rate to 1e-5
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=100,
    weight_decay=0.0001,
    logging_dir="./logs",
    logging_steps=100,
    save_total_limit=2,
    load_best_model_at_end=True
)


# Use default data collator for token classification
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=hf_dataset["train"],
    eval_dataset=hf_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Train the model
trainer.train()

# Save the model and tokenizer
model.save_pretrained("./ner_model")
tokenizer.save_pretrained("./ner_model")


Label to ID Mapping: {'B-AFFIRMATION': 0, 'B-CONFIRMATION': 1, 'B-COUNT': 2, 'B-END_DATE': 3, 'B-ORD': 4, 'B-REASON': 5, 'B-START_DATE': 6, 'I-END_DATE': 7, 'I-REASON': 8, 'I-START_DATE': 9, 'O': 10}


Map: 100%|██████████| 93/93 [00:00<00:00, 5872.08 examples/s]


Map: 100%|██████████| 11/11 [00:00<00:00, 2829.82 examples/s]
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/harshit/IITH/Sem7/CyberSecurityAndAI/cyber_security_project/chatbot_venv/lib/python3.12/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_12434/4147611080.py:74: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
  1%|          | 47/4700 [00:03<04:53, 15.87it/s]

{'eval_loss': 0.22458265721797943, 'eval_runtime': 0.0699, 'eval_samples_per_second': 157.387, 'eval_steps_per_second': 85.847, 'epoch': 1.0}


  2%|▏         | 94/4700 [00:07<04:49, 15.92it/s]

...


100%|██████████| 4700/4700 [08:17<00:00, 15.83it/s]

{'eval_loss': 0.04508901387453079, 'eval_runtime': 0.0302, 'eval_samples_per_second': 364.469, 'eval_steps_per_second': 198.801, 'epoch': 100.0}


100%|██████████| 4700/4700 [08:24<00:00,  9.32it/s]


{'train_runtime': 504.2092, 'train_samples_per_second': 18.445, 'train_steps_per_second': 9.322, 'train_loss': 0.017546459908022526, 'epoch': 100.0}


('./ner_model/tokenizer_config.json',
 './ner_model/special_tokens_map.json',
 './ner_model/vocab.txt',
 './ner_model/added_tokens.json',
 './ner_model/tokenizer.json')

In [105]:
from transformers import BertTokenizerFast, BertForTokenClassification
import torch

# Load the saved model and tokenizer
# model_dir = "./ner_model"
# tokenizer = BertTokenizerFast.from_pretrained(model_dir)
# model = BertForTokenClassification.from_pretrained(model_dir)

# Load the label mappings
id_to_label = model.config.id2label

# Sample text to test the model
test_sentences = [
    "[INT] [BOT]  [USR] Order 121435, whats the status?"
]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def predict_ner(sentence):
    # Tokenize the sentence
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True, max_length=128).to(device)
    # print(inputs)

    # Get model predictions
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Extract logits and compute predictions
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=2)

    # Map predictions to labels
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    predicted_labels = [id_to_label[pred.item()] for pred in predictions[0]]

    # Combine tokens and predicted labels
    results = []
    for token, label in zip(tokens, predicted_labels):
        if token not in ["[CLS]", "[SEP]", "[PAD]"]:  # Ignore special tokens
            results.append((token, label))
    return results

# Test the model with sample sentences
for sentence in test_sentences:
    # print(f"Sentence: {sentence}")
    predictions = predict_ner(sentence)
    # print("Predictions:")
    # for token, label in predictions:
        # print(f"{token:15} -> {label}")
    # print("\n")

def extract_entity(sentence, entity_label):
    """
    enitity_label: the label of the entity to extract (ORD for example)
    extract the entity from the sentence along with I-ORD tokens
    """
    # Get NER predictions
    predictions = predict_ner(sentence)
    # for token, label in predictions:
    #     # print(f"{token:15} -> {label}")

    # get all entities in the label
    all_entities = []
    for token, label in predictions:
        if label.startswith("B-") or label.startswith("I-"):
            all_entities.append(label.split("-")[1])

    all_entities = list(set(all_entities))

    return_values = []
    for entity_label in all_entities:
        # Extract entities with the specified label
        entities = []
        for token, label in predictions:
            if label == f"B-{entity_label}" or label == f"I-{entity_label}":
                entities.append(token)
        # print(entities)
        
        # remove subword prefixes from the entities
        entity_combined = []
        prev_entity = None
        for i , entity in enumerate(entities):
            if entity.startswith("##"):
                if prev_entity is None:
                    prev_entity = ""
                prev_entity += entity[2:]
            else:
                if prev_entity:
                    entity_combined.append(prev_entity)
                prev_entity = entity

            if i == len(entities) - 1:
                entity_combined.append(prev_entity)
        return_values.append((entity_label, entity_combined))
    return return_values
        

# get the B-COUNT entity from the sentence
sentence = "[INT] [BOT]  [USR] Please show me a list of all the orders I've made before 21 November 2024 and after 10 April 2024"
entity_label = "START_DATE"
extracted_entity = extract_entity(sentence, entity_label)
print(f"Extracted entity: {extracted_entity}")

Extracted entity: [('END_DATE', ['november44']), ('START_DATE', ['21', '202', '10', 'april', '202'])]


In [106]:
# test for 15 random samples
data = pd.read_csv('./data/ner_data.csv')

# draw 15 random samples
test_data = data.sample(15)

for idx, row in test_data.iterrows():
    print(f"Sentence: {row['instruction']}")
    # get the 
    extracted_entity = extract_entity(row['instruction'], "ORD")
    print(f"Extracted entity: {extracted_entity}")
    print("")

Sentence: [INT] [BOT]  [USR] Can you check my order 12345?
Extracted entity: [('ORD', ['12345'])]

Sentence: [INT] cancel_order [BOT] Please confirm the cancellation of your last order. [USR] Yes, confirm it.
Extracted entity: [('AFFIRMATION', ['yes'])]

Sentence: [INT] [BOT]  [USR] Cancel the last item I ordered.
Extracted entity: [('ORD', ['last'])]

Sentence: [INT] give_order_id [BOT] Are you sure? [USR] Yes I am sure!
Extracted entity: [('AFFIRMATION', ['yes'])]

Sentence: [INT] list_orders [BOT] Do you need to filter anything? [USR] Yeah give me just 32 of them
Extracted entity: [('COUNT', ['32'])]

Sentence: [INT] cancel_order [BOT] Can you confirm the cancellation of your most recent order? [USR] No, don’t proceed with it.
Extracted entity: [('AFFIRMATION', ['no'])]

Sentence: [INT] cancel_order [BOT] Are you sure you want to cancel order 112233? [USR] No, leave it as is.
Extracted entity: [('AFFIRMATION', ['no'])]

Sentence: [INT] [BOT]  [USR] Can you list some of my previous o